In [ ]:
from bloqade import move
%pip install ../assets/scorer
import math
pi = math.pi
@move.vmove
def local_hadamard(state: move.core.AtomState, indices) -> move.core.AtomState:
    state = move.LocalXY(state, 0.25 * pi, 0.5 * pi, indices)
    state = move.LocalRz(state, pi, indices)
    state = move.LocalXY(state, -0.25 * pi, 0.5 * pi, indices)
    return state

In [ ]:
@move.vmove
def problem4():
    q = move.NewQubitRegister(9)

    state = move.Init(qubits=[q[0],q[1],q[2],q[3],q[4],q[5],q[6],q[7],q[8]], indices=[0,1,2,3,4,5,6,7,8])
    state.gate[[3,6]] = move.Move(state.storage[[3,6]])
    state = local_hadamard(state,[3,6])
    state.gate[[4]] = move.Move(state.storage[[0]])
    state = move.GlobalCZ(atom_state = state)
    state.storage[[3]] = state.gate[[3]]
    state = move.GlobalCZ(atom_state = state)
    state.gate[[1,2,4,5,7,8]] = move.Move(state.storage[[1,2,4,5,7,8]])    
    state = local_hadamard(state,[1,2,4,5,7,8])
    state.storage[[2,5,8]] = move.Move(state.gate[[2,5,8]])
    state = move.GlobalCZ(atom_state = state)
    state.storage[[1,4,7]] = move.Move(state.gate[[1,4,7]])
    state.gate[[2,5,8]] = move.Move(state.storage[[2,5,8]])
    state = move.GlobalCZ(atom_state = state)
    state = local_hadamard(state,[[2,5,8]])
    state.storage[[2,5,8]] = move.Move(state.storage[[2,5,8]])
    move.Execute(state)

In [ ]:
from iquhack_scoring import MoveScorer
from kirin.passes import aggressive
expected_qasm = """
// Generated from Cirq v1.4.1

OPENQASM 2.0;
include "qelib1.inc";


// Qubits: [q(0), q(1), q(2), q(3), q(4), q(5), q(6), q(7), q(8)]
qreg q[9];


cx q[0],q[3];
cx q[0],q[6];
h q[3];
h q[0];
h q[6];
cx q[3],q[4];
cx q[0],q[1];
cx q[6],q[7];
cx q[3],q[5];
cx q[0],q[2];
cx q[6],q[8];
"""
analysis = MoveScorer(problem5,expected_qasm = expected_qasm)

MoveScorer(problem5, expected_qasm=expected_qasm).animate()
print(MoveScorer(problem5, expected_qasm=expected_qasm).score())